Name - Meeran Siddiqui

net id - mxs220072

HW 4 – Multi-label Classification Using Trainer (10 Points)

Required Submission files:

Submit the following two files:
1.	FirstName_LastName_HW4.ipynb
2.	FirstName_LastName_HW4.pdf (pdf version of the above file)



TASK: Multilabel Classification (10 Points)

In this HW, you will identify tags for stack exchange Questions. This data is a subset of data available in Kaggle Competitions. The given dataset actually contains the different questions asked in the StackExchange website for various tech domains. We have fetched only those question that contains top 10 individual tags. Each question can have more than one tag. This means that this is a multi-label classification problem. These are the ten categories for tags in the data.

0	c#
1	java
2	php
3	javascript
4	android
5	jquery
6	c++
7	python
8	iphone
9	asp.net

Data:
•	The dataset is available from eLearning :0_Data folder from the file: df_multilabel_hw_cleaned.joblib.
•	The dataset is a pandas dataframe. You can use joblib.load(file) to load the dataset.
•	No preprocessing is required, you are provided with the cleaned dataset.
•	You will use this data set to train your classification model.
•	Hint1: The Tag_Number column is your labels. However, the data type of list elements is object. To convert it to integers;use  ast.literatl_eval() function from ast library. You will need to import this library first.
•	Hint2: Since this is a multilabel dataset – you will need to do the one-hot encoding of your dependent variable. You can use MultiLabelBinarizer from sklearn.
•	You will create train/valid/test splits from this data set. Use 60% for train, 20 % for the test, and the remaining 20% for the validation dataset.

Model:
●	Your Network should have the following layers

EmbeddingBag → Hidden_Layer1 →ReLU  → Dropout_Layer1 → BatchNorn_Layer1 → Hidden_Layer2 → ReLU  → DropoutLayer2 → BatchNorm_Layer2  → Output Layer
•	You will use the following hyperparameters to train our model:
    EMBED_DIM=300,
    VOCAB_SIZE=len(your_vocab),
    OUTPUT_DIM=10,
    HIDDEN_DIM1=200,
    HIDDEN_DIM2=100,
        
    # training
    EPOCHS=5,
    BATCH_SIZE=128,
    LEARNING_RATE=0.001,
    WEIGHT_DECAY=0.000,
    CLIP_TYPE = 'value',
    CLIP_VALUE = 10,
    PATIENCE=5,

Note:  For Gradient Clipping. you will need to add the following line in the step function we discussed in class.
loss.backward()

# Clip gradients before the optimizer step
clip_grad_value_(model.parameters(), clip_value=10.0)

# Update parameters
optimizer.step()

Instead of accuracy use Hamming Loss as a metric, since this is a multilabel classification problem. Hint (see Task 4 of HW3 for reference).

You can use 3_imdb_complete_example.ipynb from Lecture 3 as a starting point. You will need to make changes to the Training Function to adapt it to the multilabel Classification problem.


# <Font color = 'indianred'>**Load Libraries/Install Software**

In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install torchtext --upgrade -qq
    !pip install torchmetrics -qq
    !pip install torchinfo -qq

    basepath = '/content/drive/MyDrive/NLP'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 8.2 MB/s eta 0:00:00


In [2]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn
import torch.nn.functional as F

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab
from torchinfo import summary

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np
import pandas as pd

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from types import SimpleNamespace
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


# <Font color = 'indianred'>**Specify Project Folders**

In [3]:
data = joblib.load('/content/drive/MyDrive/NLP/df_multilabel_hw_cleaned.joblib')
data.head(10)

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"[0, 9]"
1,run javascript code server java code want run ...,java javascript,"[1, 3]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[0, 9]"
3,run python script php server run nginx web ser...,php python,"[2, 7]"
4,advice write function m try write function res...,javascript jquery,"[3, 5]"
5,jquery auto resize function cause jump browser...,javascript jquery,"[3, 5]"
6,php page redirect operation page php grid subp...,php javascript,"[2, 3]"
7,advice need expert asp.net usercontrol usage n...,c# asp.net,"[0, 9]"
8,revert style apply focus blur user focus text ...,javascript jquery,"[3, 5]"
9,hack work look android source develop app app ...,java android,"[1, 4]"


In [4]:
import ast
for i in range (len(data['Tag_Number'])):
  data['Tag_Number'][i] = ast.literal_eval(data['Tag_Number'][i])

## Multi label one hot encoding

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
x=mlb.fit_transform(data['Tag_Number'])

for i in range (len(data['Tag_Number'])):
  data['Tag_Number'][i] = x[i]

In [6]:
data.head()

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,run javascript code server java code want run ...,java javascript,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,run python script php server run nginx web ser...,php python,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]"
4,advice write function m try write function res...,javascript jquery,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"


In [7]:
data.drop(['Tags'],axis=1,inplace=True)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_percentage = 0.6
validation_percentage = 0.2
test_percentage = 0.2

df_train, df_temp = train_test_split(data, test_size=1 - train_percentage, random_state=42)


relative_test_percentage = test_percentage / (validation_percentage + test_percentage)
df_validation, df_test = train_test_split(df_temp, test_size=relative_test_percentage, random_state=42)


In [9]:
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

(28456, 2)
(9485, 2)
(9486, 2)


In [10]:
df_train.head(10)

,cleaned_text,Tag_Number
21010,rearrange order list web page base current day...,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 0]"
14838,drop default selection cause search begin sear...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
13376,send javascript variable php send value store ...,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
39416,check session expired ve specify session time ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
15883,pass value server control jquery update code c...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]"
26175,pass dymanic tablelayout xml form linearlayout...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
47235,datum addition updation sql table iam fairly n...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
18819,iterate loop loop base array good way ignore d...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
14841,android unregisterreceiver onpause like manife...,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0]"
34798,get error exist write follow code refer work...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [11]:
X_train = df_train['cleaned_text']
y_train = df_train['Tag_Number']

X_val = df_validation['cleaned_text']
y_val = df_validation['Tag_Number']

X_test = df_test['cleaned_text']
y_test = df_test['Tag_Number']

In [12]:
X_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 28456 entries, 21010 to 15795
Series name: cleaned_text
Non-Null Count  Dtype 
--------------  ----- 
28456 non-null  object
dtypes: object(1)
memory usage: 444.6+ KB


## <Font color = 'indianred'>**Custom Dataset Class**

In [13]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Custom Dataset class for loading IMDB reviews and labels.

    Attributes:
        X (numpy.ndarray): Feature data, an array of texts.
        y (list or array-like): Target labels.
    """

    def __init__(self, X, y):
        """
        Initialize the dataset with feature and target data.

        Args:
            X (list or array-like): The feature data (texts).
            y (list or array-like): The target labels.
        """
        # Converting the feature data to a NumPy array for consistency
        self.X = np.array(X)

        # Storing the target labels
        self.y = y

    def __len__(self):
        """
        Return the number of samples in the dataset.

        Returns:
            int: The total number of samples.
        """
        return len(self.X)

    def __getitem__(self, idx):
        """
        Fetch and return a single sample from the dataset at the given index.

        Args:
            idx (int): Index of the sample to fetch.

        Returns:
            tuple: A tuple containing the label and the text for the sample.
        """
        # Retrieve the text and corresponding label from the dataset using the index
        texts = self.X[idx]
        labels = self.y[idx]

        # Packing them into a tuple before returning
        sample = (labels, texts)

        return sample


In [14]:
trainset = CustomDataset(X_train.values,y_train.values)
validset = CustomDataset(X_val.values ,y_val.values )
testset = CustomDataset(X_test.values,y_test.values )

## <Font color = 'indianred'>**Create Vocab**

In [15]:
def get_vocab(dataset, min_freq=1):
    """
    Generate a vocabulary from a dataset.

    Args:
        dataset (list of tuple): List of tuples where each tuple contains a label and a text.
        min_freq (int): The minimum frequency for a token to be included in the vocabulary.

    Returns:
        torchtext.vocab.Vocab: Vocabulary object.
    """
    # Initialize a counter object to hold token frequencies
    counter = Counter()

    # Update the counter with tokens from each text in the dataset
    for (l_, text) in dataset:
        counter.update(str(text).split())

    # Create a vocabulary using the counter object
    # Tokens that appear fewer times than `min_freq` are excluded
    my_vocab = vocab(counter, min_freq=min_freq)

    # Insert a '<unk>' token at index 0 to represent unknown words
    my_vocab.insert_token('<unk>', 0)

    # Set the default index to 0
    # This ensures that any unknown word will be mapped to '<unk>'
    my_vocab.set_default_index(0)

    return my_vocab


vocab should always be created based on trainset

In [16]:
vocab = get_vocab(trainset, min_freq=2)

In [17]:
len(vocab)

90287

In [18]:
vocab.get_itos()[0:5]

['<unk>', 'rearrange', 'order', 'list', 'web']

## <Font color = 'indianred'>**Collate_fn for Data Loaders**

In [20]:
# Creating a function that will be used to get the indices of words from vocab
def text_pipeline(x, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(x).split()]

In [21]:
def collate_batch(batch, my_vocab):
    """
    Collates a batch of samples into tensors of labels, texts, and offsets.

    Parameters:
        batch (list): A list of tuples, each containing a label and a text.

    Returns:
        tuple: A tuple containing three tensors:
               - Labels tensor
               - Concatenated texts tensor
               - Offsets tensor indicating the start positions of each text in the concatenated tensor
    """
    # Unpack the batch into separate lists for labels and texts
    labels, texts = zip(*batch)

    # Convert the list of labels into a tensor of dtype int32
    labels = torch.tensor(labels, dtype=torch.long)

    # Convert the list of texts into a list of lists; each inner list contains the vocabulary indices for a text
    list_of_list_of_indices = [text_pipeline(text, my_vocab) for text in texts]

    # Compute the offsets for each text in the concatenated tensor
    offsets = [0] + [len(i) for i in list_of_list_of_indices]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

    # Concatenate all text indices into a single tensor
    texts = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])

    return (texts, offsets), labels

## <Font color = 'indianred'>**Check Data Loaders**

Let us check if our collate function is working by creating a dataloader

In [22]:
batch_size = 2
check_loader = torch.utils.data.DataLoader(dataset=trainset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=partial(collate_batch, my_vocab = vocab),
                                           )

In [23]:
torch.manual_seed(22)
for (text, offset), label in check_loader:
    print(text, offset, label)
    break


tensor([   96,   128,  3259,  1141,  1009,   597,   768,   200,   521,   563,
           24,  1691,   563,     5,   439,  1755,   377,  3259,   563,   768,
          522, 24247,    31,   687,   563,   464,  2527,   614,  1141,   190,
         2287,   508,   492,   464,  2527,   563,  1275,   508,   101,   614,
          506,     0,     0,     0,  1475,  1573,   563,   101,   614,   231,
          546, 28690,   114,   333,   325,   847, 47780,    88,  3245,   173,
        60429,    50,   205,   231,  1561,  3245,   173,    24,   461,  1561,
          173, 41607,  1151,   204,   231,  1561,    81,   173,   805,   159,
          427,    81,   427,   175,   427,   428,   427,   508,   427,   509,
          427,   511,   204,    76,   452,   439,  5581,    81,    81,    87,
         4607, 47756,   191,   546,   166,  7549,    81, 19582, 35273,  4606,
          767, 32621, 35274,  1682,   767, 32617, 35275, 35277, 35278,  1682,
          767, 32618, 35279, 35280, 35278,  1682,   767, 32619, 

<ipython-input-21-d54c96152e61>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  labels = torch.tensor(labels, dtype=torch.long)


# <Font color = 'indianred'>**Model**
embedding_layer-->linear--> ReLU -->dropout --> batch norm --> linear-->ReLu-->Dropout--> batchnorm ---> linear layer

In [24]:
class CustomBlock(nn.Module):
    def __init__(self, input_dim, output_dim):

        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Dropout(),
            nn.BatchNorm1d(num_features=output_dim),


        )
    def forward(self, x):
      return self.layers(x)



## <Font color = 'indianred'>**Model with EmbeddingBag layer**





## <Font color = 'indianred'>**Model with EmbeddingBag layer - Option1**

## <Font color = 'indianred'>**Model with EmbeddingBag layer - Option2**

In [25]:
class CustomMLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, num_outputs):
        super().__init__()

        self.embedding_bag = nn.EmbeddingBag(vocab_size, embedding_dim)
        self.layers = nn.Sequential(
            CustomBlock(embedding_dim , hidden_dim1),
            CustomBlock(hidden_dim1, hidden_dim2),
            nn.Linear(hidden_dim2, num_outputs)
        )

    def forward(self, input_tuple):
        data, offsets = input_tuple
        embed_out = self.embedding_bag(data, offsets)
        out = self.layers(embed_out)

        return out

# <Font color = 'indianred'>**Model Training Functions**




To train a machine learning model effectively, we need a structured approach that combines training and evaluation while maintaining flexibility and readability. We break down the training loop into four distinct functions, each serving a specific role. This modular approach enhances code clarity, reusability, and allows for easy modifications. Let's outline our strategy for training and evaluation:

1. **`step` Function**: This is our foundational building block. It handles a single forward and backward step for a batch of data. Whether during training or evaluation, we need to calculate model outputs, compute the loss, and potentially perform backpropagation to update model parameters. By encapsulating these fundamental operations in `step`, we ensure consistency and enable future adjustments with minimal disruption.

  **Model Training** involves five steps:

- Step 1: Compute model's predictions - forward pass (both train and eval)
- Step 2: Compute loss (both train and eval)
- Step 3: Compute the gradients (only train)
- Step 4: Update the parameters (only train)
- Step 5: Repeat steps 1 - 4


2. **`train_epoch` Function**: This function orchestrates training for a single epoch. It iterates through the entire training dataset, invoking `step` for each batch. While doing so, it keeps track of the running training loss and the number of correct predictions. Its primary role is to aggregate statistics for the entire epoch. Additionally, it ensures that the model is in the training mode.

3. **`val_epoch` Function**: Similar to `train_epoch`, this function handles validation. It iterates through the validation dataset, employing `step` for each batch, but crucially, it operates in a context where gradients aren't computed. This no-gradient mode conserves memory and computational resources. Like `train_epoch`, it is responsible for aggregating validation metrics and also ensures that the model is in evaluation mode.

4. **`train` Function**: Our top-level function for training the model. It sets up the training environment for multiple epochs, invoking `train_epoch` for each training epoch and `val_epoch` for each validation epoch. It keeps track of metrics across epochs, making it easier to analyze model performance over time.

By organizing our training and evaluation pipeline in this manner, we achieve code modularity and clarity. Furthermore, this structure allows us to easily swap out components like loss functions, optimizers, or metrics without overhauling the entire codebase. Now, let's dive into the details of each function.

## <Font color = 'indianred'>**Step Function**

In [26]:
def step(inputs, targets, model, device, loss_function=None, optimizer=None):
    """
    Performs a forward and backward pass for a given batch of inputs and targets.

    Parameters:
    - inputs (torch.Tensor): The input data for the model.
    - targets (torch.Tensor): The true labels for the input data.
    - model (torch.nn.Module): The neural network model.
    - device (torch.device): The computing device (CPU or GPU).
    - loss_function (torch.nn.Module, optional): The loss function to use.
    - optimizer (torch.optim.Optimizer, optional): The optimizer to update model parameters.

    Returns:
    - loss (float): The computed loss value (only if loss_function is not None).
    - outputs (torch.Tensor): The predictions from the model.
    - correct (int): The number of correctly classified samples in the batch.
    """
    # Move the model and data to the device
    model = model.to(device)
    inputs = tuple(input_tensor.to(device)
                            for input_tensor in inputs)

    targets = targets.to(device)
    targets = targets.to(dtype=torch.float32)
    #print("output shape",targets.shape)

    # Step 1: Forward pass to get the model's predictions
    outputs = model(inputs)
    #print("output shape",output.shape)

    # Step 2: Compute the loss using the provided loss function
    if loss_function:
        loss = loss_function(outputs, targets)

    # Step 2: Calculate the number of correctly classified samples
    #predicted = torch.argmax(outputs.data, 1)
    correct = (outputs == targets).sum().item()

    # Step 3 and 4: Perform backward pass and update model parameters if an optimizer is provided
    if optimizer:
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=10.0)
        optimizer.step()

    # Return relevant metrics
    if loss_function:
        return loss, outputs, correct
    else:
        return outputs, correct


## <Font color = 'indianred'>**Train_Epoch Function**

In [27]:
def train_epoch(train_loader, model, device, loss_function, optimizer):
    """
    Trains the model for one epoch using the provided data loader and updates the model parameters.

    Parameters:
    - train_loader (torch.utils.data.DataLoader): DataLoader object for the training set.
    - model (torch.nn.Module): The neural network model to be trained.
    - device (torch.device): The computing device (CPU or GPU).
    - loss_function (torch.nn.Module): The loss function to use for training.
    - optimizer (torch.optim.Optimizer): The optimizer to update model parameters.

    Returns:
    - train_loss (float): Average training loss for the epoch.
    - train_acc (float): Training accuracy for the epoch.
    """
    train_hamming_distance = HammingDistance(task="multilabel", num_labels=10).to(device)
    # Set the model to training mode
    model.train()

    # Initialize variables to track running training loss and correct predictions
    running_train_loss = 0.0
    running_train_correct = 0

    # Iterate over all batches in the training data
    for inputs, targets in train_loader:
        # Perform a forward and backward pass, updating model parameters
        loss, _, correct = step(inputs, targets, model, device, loss_function, optimizer)

        # Update running loss and correct predictions counter
        running_train_loss += loss.item()
        running_train_correct += correct

    # Compute average loss and accuracy for the entire training set
    train_loss = running_train_loss / len(train_loader)
    train_acc = running_train_correct / len(train_loader.dataset)

    epoch_hamming_distance = train_hamming_distance.compute()

    # # Print the train loss and Hamming Distance for the epoch
    # print(f'Epoch: {epoch + 1} / {epochs}')
    print(f'Train Loss: {train_loss:.4f} | Train Hamming Distance: {epoch_hamming_distance:.4f}')

    # # Reset metric states for the next epoch
    # train_hamming_distance.reset()


    return train_loss, train_acc


## <Font color = 'indianred'>**Val_Epoch Function**

In [28]:
def val_epoch(valid_loader, model, device, loss_function):
    """
    Validates the model for one epoch using the provided data loader.

    Parameters:
    - valid_loader (torch.utils.data.DataLoader): DataLoader object for the validation set.
    - model (torch.nn.Module): The neural network model to be validated.
    - device (torch.device): The computing device (CPU or GPU).
    - loss_function (torch.nn.Module): The loss function to evaluate the model.

    Returns:
    - val_loss (float): Average validation loss for the epoch.
    - val_acc (float): Validation accuracy for the epoch.
    """
    #train_hamming_distance = HammingDistance(task="multilabel", num_labels=10).to(device)
    # Set the model to evaluation mode
    model.eval()

    # Initialize variables to track running validation loss and correct predictions
    running_val_loss = 0.0
    running_val_correct = 0

    # Disable gradient computation
    with torch.no_grad():
        # Iterate over all batches in the validation data
        for inputs, targets in valid_loader:
            # Perform a forward pass to get loss and number of correct predictions
            loss, _, correct = step(inputs, targets, model, device, loss_function, optimizer=None)

            # Update running loss and correct predictions counter
            running_val_loss += loss.item()
            running_val_correct += correct

    # Compute average loss and accuracy for the entire validation set
    val_loss = running_val_loss / len(valid_loader)
    val_acc = running_val_correct / len(valid_loader.dataset)

    # epoch_hamming_distance = train_hamming_distance.compute()

    # # Print the train loss and Hamming Distance for the epoch
    # print(f'Epoch: {epoch + 1} / {epochs}')
    # print(f'Train Loss: {train_loss:.4f} | Train Hamming Distance: {epoch_hamming_distance:.4f}')

    # # Reset metric states for the next epoch
    # train_hamming_distance.reset()

    return val_loss, val_acc


## <Font color = 'indianred'>**Train Function**

In [29]:
from torchmetrics import HammingDistance

In [30]:
def train(train_loader, valid_loader, model, optimizer, loss_function, epochs, device):
    """
    Trains and validates the model, and returns history of train and validation metrics.

    Parameters:
    - train_loader (torch.utils.data.DataLoader): DataLoader for the training set.
    - valid_loader (torch.utils.data.DataLoader): DataLoader for the validation set.
    - model (torch.nn.Module): Neural network model to train.
    - optimizer (torch.optim.Optimizer): Optimizer algorithm.
    - loss_function (torch.nn.Module): Loss function to evaluate the model.
    - epochs (int): Number of epochs to train the model.
    - device (torch.device): The computing device (CPU or GPU).

    Returns:
    - train_loss_history (list): History of training loss for each epoch.
    - train_acc_history (list): History of training accuracy for each epoch.
    - valid_loss_history (list): History of validation loss for each epoch.
    - valid_acc_history (list): History of validation accuracy for each epoch.
    """
    train_hamming_distance = HammingDistance(task="multilabel", num_labels=10).to(device)
    # Initialize lists to store metrics for each epoch
    train_loss_history = []
    valid_loss_history = []
    train_acc_history = []
    valid_acc_history = []

    # Loop over the number of specified epochs
    for epoch in range(epochs):
      # Train model on training data and capture metrics
      train_loss, train_acc = train_epoch(
          train_loader, model, device, loss_function, optimizer)

        # Validate model on validation data and capture metrics
      valid_loss, valid_acc = val_epoch(
          valid_loader, model, device, loss_function)

        # Store metrics for this epoch
      train_loss_history.append(train_loss)
      train_acc_history.append(train_acc)
      valid_loss_history.append(valid_loss)
      valid_acc_history.append(valid_acc)

        # Output epoch-level summary
      print(f"Epoch {epoch+1}/{epochs}")
      print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc*100:.2f}%")
      print(f"Valid Loss: {valid_loss:.4f} | Valid Accuracy: {valid_acc*100:.2f}%")
      print()

      epoch_hamming_distance = train_hamming_distance.compute()

      # Print the train loss and Hamming Distance for the epoch
      print(f'Epoch: {epoch + 1} / {epochs}')
      print(f'Train Loss: {train_loss:.4f} | Train Hamming Distance: {epoch_hamming_distance:.4f}')

      # Reset metric states for the next epoch
      train_hamming_distance.reset()

    return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history


# <Font color = 'indianred'>**Hyperparameters**

In [31]:
hyperparameters = SimpleNamespace(
    # model Parameters
    EMBED_DIM=300,
    VOCAB_SIZE=len(vocab),
    OUTPUT_DIM=10,
    HIDDEN_DIM1=200,
    HIDDEN_DIM2=100,
    #DROP_PROB1=0.5,
    #DROP_PROB2=0.5,
    NUM_OUTPUTS=10,

    # training
    EPOCHS=5,
    BATCH_SIZE=128,
    LEARNING_RATE=0.001,
    WEIGHT_DECAY=0.000,
    CLIP_TYPE = 'value',
    CLIP_VALUE = 10,
    PATIENCE=5,


)

# <Font color = 'indianred'>**Training Configuration**

In [32]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

collate_fn = partial(collate_batch, my_vocab=vocab)

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=hyperparameters.BATCH_SIZE, shuffle=True,
                                           collate_fn=collate_fn, num_workers=4)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=hyperparameters.BATCH_SIZE, shuffle=False,
                                           collate_fn=collate_fn,  num_workers=4)
test_loader = torch.utils.data.DataLoader(testset, batch_size=hyperparameters.BATCH_SIZE,   shuffle=False,
                                          collate_fn=collate_fn,  num_workers=4)

# cross entropy loss function
loss_function = nn.CrossEntropyLoss()
#loss_function = nn.MSELoss()

# model
model_imdb = CustomMLP(vocab_size=hyperparameters.VOCAB_SIZE,
                       embedding_dim=hyperparameters.EMBED_DIM,
                       hidden_dim1=hyperparameters.HIDDEN_DIM1,
                       hidden_dim2=hyperparameters.HIDDEN_DIM2,
                       num_outputs=hyperparameters.NUM_OUTPUTS)

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)


# apply initialization recursively  to all modules
model_imdb.apply(init_weights)

# Intialize stochiastic gradient descent optimizer
optimizer = torch.optim.AdamW(model_imdb.parameters(),
                             lr=hyperparameters.LEARNING_RATE,

                             )

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# <Font color = 'indianred'>**Sanity Check**
- Check the loss without any training. For Cross entropy the expected value will be log(number of classes)

In [33]:
for input_ , targets  in train_loader:
  # move inputs and outputs to GPUs
  model_imdb = model_imdb.to(device)

  input_ = tuple(input_tensor.to(device)
                            for input_tensor in input_)
  targets = targets.to(device)
  print(type(targets))
  targets = targets.to(dtype=torch.float32)
  model_imdb.eval()
  # Forward pass
  output = model_imdb(input_)
  print(type(output))
  loss = loss_function(output, targets)
  print(f'Actual loss: {loss}')
  break

print(f'Expected Theoretical loss: {np.log(10)}')

<class 'torch.Tensor'>
<class 'torch.Tensor'>
Actual loss: 4.873618125915527
Expected Theoretical loss: 2.3025850929940455


# <Font color = 'indianred'>**Training Model**

In [34]:
# Call the train function to train the model
train_losses, train_acc, valid_losses, valid_acc = train(
    train_loader, valid_loader, model_imdb, optimizer, loss_function, hyperparameters.EPOCHS, device
)


Train Loss: 3.8004 | Train Hamming Distance: 1.0000


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MultilabelHammingDistance was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 1/5
Train Loss: 3.8004 | Train Accuracy: 0.00%
Valid Loss: 2.6303 | Valid Accuracy: 0.00%

Epoch: 1 / 5
Train Loss: 3.8004 | Train Hamming Distance: 1.0000
Train Loss: 2.6755 | Train Hamming Distance: 1.0000
Epoch 2/5
Train Loss: 2.6755 | Train Accuracy: 0.00%
Valid Loss: 2.4210 | Valid Accuracy: 0.00%

Epoch: 2 / 5
Train Loss: 2.6755 | Train Hamming Distance: 1.0000
Train Loss: 2.4598 | Train Hamming Distance: 1.0000
Epoch 3/5
Train Loss: 2.4598 | Train Accuracy: 0.00%
Valid Loss: 2.3265 | Valid Accuracy: 0.00%

Epoch: 3 / 5
Train Loss: 2.4598 | Train Hamming Distance: 1.0000
Train Loss: 2.3312 | Train Hamming Distance: 1.0000
Epoch 4/5
Train Loss: 2.3312 | Train Accuracy: 0.00%
Valid Loss: 2.2540 | Valid Accuracy: 0.00%

Epoch: 4 / 5
Train Loss: 2.3312 | Train Hamming Distance: 1.0000
Train Loss: 2.2363 | Train Hamming Distance: 1.0000
Epoch 5/5
Train Loss: 2.2363 | Train Accuracy: 0.00%
Valid Loss: 2.2050 | Valid Accuracy: 0.00%

Epoch: 5 / 5
Train Loss: 2.2363 | Train Hamming